In [1]:
%matplotlib inline

import sys
from lumapi import MODE
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

C:\Program Files\Lumerical\v221\api\python\lumapi.py:796: SyntaxWarning: invalid escape sequence '\s'
  message = re.sub('^(Error:)\s(prompt line)\s[0-9]+:', '', str(rvals[2])).strip()


In [2]:
mode = MODE()

In [3]:
um = 1e-6
nm = 1e-9

## Definindo Geometria

In [ ]:
sub_material = 'SiO2 (Glass) - Palik'

comprimento = 8*um
largura = 0.45*um
altura = 0.22*um
diametro = 8*um
guia_inferior = 50*um
gap = 0.2*um

core_material = 'Si (Silicon) - Palik'


In [16]:
mode.switchtolayout()
mode.deleteall()

mode.addrect()
mode.set('name', 'core_1')
mode.set('material', core_material)
mode.set("x",0)
mode.set("y",0)
mode.set("z",0)
mode.set("x span",2*comprimento)
mode.set("y span",largura)
mode.set("z span",altura)

mode.addobject("180_bend_wg")
mode.set("name", "bend")
mode.set("first axis", 'z')
mode.set("base width", largura)
mode.set("base height", altura)
mode.set("radius", diametro)
mode.set("rotation 1", 180)
mode.set("x", 0)
mode.set("y", diametro + largura + gap)
mode.set("z", 0)



## Definindo Geometria do Solver

In [22]:
fde_solve_material = sub_material

x_span_fde = 0
y_span_fde = 3*um
z_span_fde = 3*um

mesh_cells = 100
wavelength = 1550*nm
startWvg = 1500*nm
stopWvg = 1600*nm
modes = 4

y_fde = (largura+gap)/2

mesh_multiplier = 2

In [ ]:
mode.switchtolayout()

mode.select("FDE")
mode.delete()

mode.select("mesh")
mode.delete()

mode.addfde()
mode.set("solver type", '2D X normal')
mode.set("background material", fde_solve_material)

mode.set("x", 0)
mode.set("y", y_fde)
mode.set("z", 0)

mode.set("y span", y_span_fde)
mode.set("z span", z_span_fde)

mode.set("define y mesh by", 'number of mesh cells')
mode.set("define z mesh by", 'number of mesh cells')
mode.set("mesh cells y", mesh_cells)
mode.set("mesh cells z", mesh_cells)

# Set solver properties
mode.set("wavelength", wavelength)

# Add ports
mode.set("number of trial modes", modes)

# Set simulation boundaries
mode.set('start waveLength', startWvg)
mode.set('stop waveLength', stopWvg)

mode.set('y min bc', 1)
mode.set('y max bc', 1)
mode.set('z min bc', 1)
mode.set('z max bc', 1)

mode.addmesh()
mode.set("set mesh multiplier", True)
mode.set("x", 0)
mode.set("y", y_fde)
mode.set("z", 0)

mode.set("x span", x_span_fde)
mode.set("y span", y_span_fde)
mode.set("z span", z_span_fde)
mode.set("y mesh multiplier", mesh_multiplier)
mode.set("z mesh multiplier", mesh_multiplier)

In [24]:
mode.findmodes()

mode.save("Guia Retangular")

## Índice de grupo e Índice efetivo

In [26]:
group_index = mode.getresult("FDE::data::mode1", "ng")
neff = mode.getresult("FDE::data::mode1", "neff")
print("ng =", np.abs(group_index[0][0]),"\nneff = ", np.abs(neff[0][0]))

ng = 4.2122121112425885 
neff =  2.369406914814772


## Para o cálculo de $L_r$

$FSR = \frac{\lambda^2}{n_g(\lambda){L_r}}$

In [27]:
FSR = np.array([1*nm,10*nm,20*nm])

Lr = (wavelength**2)/(np.abs(group_index[0][0])*FSR)
print(Lr*1e6)

[570.36538915  57.03653891  28.51826946]


## Acoplador Direcional fornecido pelo PDK SciPIC

- **Lc** = 0 → Comprimento de acoplamento  
- **Gap** = 0.2 µm  
- **Raio** = 18 µm  
- **Espessura** = 0.22 µm  
- **Largura** = 0.5 µm  

In [45]:
# Raio do acoplador

raio_dc = Lr[2]/(2*np.pi)
# Comprimento total dc

print(f'Raio do acoplador DC = {raio_dc * 1e6:.5f} um')

comprimento_guias = Lr - (2*np.pi*raio_dc)
print('Comprimentos para cada guia: ',(comprimento_guias*1e6)/2,' um')


Raio do acoplador DC = 4.53882 um
Comprimentos para cada guia:  [270.92355984  14.25913473   0.        ]  um
